In [ ]:
import json
import sys
from dotenv import load_dotenv, find_dotenv
import os

sys.path.append('../../aisuite')

# Load from .env file if available
load_dotenv(find_dotenv())

### Define the function

In [2]:
def will_it_rain(location: str, time_of_day: str):
    """Check if it will rain in a location at a given time today.

    Args:
        location (str): Name of the city
        time_of_day (str): Time of the day in HH:MM format.
    """
    return "YES"

---

### Using OpenAI

### JSON spec for the function

In [4]:
tools = [{
    "type": "function",
    "function": {
        "name": "will_it_rain",
        "description": "Check if it will rain in a location at a given time today",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Name of the city"
                },
                "time_of_day": {
                    "type": "string",
                    "description": "Time of the day in HH:MM format."
                }
            },
            "required": ["location", "time_of_day"]
        }
    }
}]

In [9]:
from openai import OpenAI

client = OpenAI()
messages = [{
    "role": "user",
    "content": "I live in San Francisco. Can you check for weather "
               "and plan an outdoor picnic for me at 2pm?"
}]
response = client.chat.completions.create(
    model="gpt-4o", messages=messages, tools=tools
)

### Process tool call response & Execute the tool

In [ ]:
response2 = None
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)

    result = will_it_rain(args["location"], args["time_of_day"])
    messages.append(response.choices[0].message)
    messages.append({
        "role": "tool", "tool_call_id": tool_call.id, "content": str(result)
    })

    response2 = client.chat.completions.create(
        model="gpt-4o", messages=messages, tools=tools)
    print(response2.choices[0].message.content)

### Optionally, continue the conversation

In [ ]:
messages.append(response2.choices[0].message)
messages.append({
    "role": "user", "content": "Schedule it despite the rain"
})
response = client.chat.completions.create(
    model="openai:gpt-4o", messages=messages)
print(response.choices[0].message.content)

---

### Using aisuite

In [ ]:
from aisuite import Client

client = Client()
messages = [{
    "role": "user",
    "content": "I live in San Francisco. Can you check for weather "
               "and plan an outdoor picnic for me at 2pm?"
}]
response = client.chat.completions.create(
    model="openai:gpt-4o", messages=messages, tools=[will_it_rain], max_turns=2
)
print(response.choices[0].message.content)

### Optionally, continue the conversation

In [ ]:
messages.extend(response.choices[0].intermediate_messages)
messages.append({
    "role": "user", "content": "Schedule it despite the rain"
})
response = client.chat.completions.create(
    model="openai:gpt-4o", messages=messages)
print(response.choices[0].message.content)

### Using, other providers

In [ ]:
messages = [{
    "role": "user",
    "content": "I live in San Francisco. Can you check for weather "
               "and plan an outdoor picnic for me at 2pm?"
}]
response = client.chat.completions.create(
    model="anthropic:claude-3-5-sonnet-20240620", messages=messages, tools=[will_it_rain], max_turns=2
)
print(response.choices[0].message.content)

In [ ]:
response = client.chat.completions.create(
    model="groq:llama3-8b-8192", messages=messages, tools=[will_it_rain], max_turns=2)